# Fish-Speech-1.5 日本語テキスト→音声変換アプリ（オフライン版）

このノートブックはFish-Speech-1.5モデルをダウンロードして、オフラインで使用します。

## 1. 準備: ライブラリのインストール
以下のセルを実行して、必要なライブラリをインストールします。初回のみ実行してください。

In [ ]:
# 必要なライブラリのインストール（最初に一度だけ実行）
!pip install torch==2.0.1 torchaudio==2.0.2 numpy scipy ipywidgets
!pip install huggingface_hub soundfile
!pip install transformers==4.31.0
!pip install onnxruntime

## 2. ライブラリのインポート

In [ ]:
import numpy as np
import torch
import torchaudio
import os
import io
import tempfile
import time
from IPython.display import Audio, display, HTML
import ipywidgets as widgets
import scipy.io.wavfile as wavfile
import soundfile as sf
from huggingface_hub import hf_hub_download
import onnxruntime as ort
from transformers import AutoProcessor, AutoTokenizer

## 3. モデルとプロセッサのダウンロード

Hugging Faceからモデルとプロセッサをダウンロードします。

In [ ]:
# モデルのダウンロード
def download_fish_speech_model(cache_dir="./models"):
    """Fish-Speech-1.5モデルをダウンロードする"""
    # キャッシュディレクトリを作成
    os.makedirs(cache_dir, exist_ok=True)
    
    print("モデルファイルをダウンロードしています...")
    
    # モデルファイルをダウンロード
    model_files = {
        "config": hf_hub_download(repo_id="fishaudio/fish-speech-1.5", filename="config.json", cache_dir=cache_dir),
        "tokenizer": hf_hub_download(repo_id="fishaudio/fish-speech-1.5", filename="tokenizer_config.json", cache_dir=cache_dir),
        "vocab": hf_hub_download(repo_id="fishaudio/fish-speech-1.5", filename="vocab.json", cache_dir=cache_dir),
        "model": hf_hub_download(repo_id="fishaudio/fish-speech-1.5", filename="pytorch_model.bin", cache_dir=cache_dir, resume_download=True)
    }
    
    # ONNXモデルも試してみる（代替として）
    try:
        model_files["onnx"] = hf_hub_download(repo_id="fishaudio/fish-speech-1.5", filename="model.onnx", cache_dir=cache_dir, resume_download=True)
    except Exception as e:
        print(f"ONNXモデルのダウンロードに失敗しました: {e}")
        model_files["onnx"] = None
    
    print("ダウンロードが完了しました。")
    return model_files

# トークナイザーの初期化
def initialize_tokenizer(cache_dir="./models"):
    try:
        tokenizer = AutoTokenizer.from_pretrained("fishaudio/fish-speech-1.5", cache_dir=cache_dir)
        return tokenizer
    except Exception as e:
        print(f"トークナイザーの初期化に失敗しました: {e}")
        return None

# モデルをダウンロード
model_files = download_fish_speech_model()

# トークナイザーを初期化
tokenizer = initialize_tokenizer()

## 4. シンプルな音声合成関数の定義

In [ ]:
# シンプルな音声合成関数
def simple_text_to_speech(text, language="ja", speed=1.0):
    """
    シンプルな音声合成関数（トランスフォーマーベース）
    
    この関数は、テキストを受け取り、簡易的な音声を生成します。
    実際のFish-Speechモデルの完全実装ではなく、デモとして機能します。
    
    Parameters:
    -----------
    text : str
        変換するテキスト
    language : str
        言語コード（'ja', 'en', 'zh'など）
    speed : float
        速度調整率（1.0が標準）
        
    Returns:
    --------
    str
        生成された音声ファイルのパス
    """
    # 一時ファイルを作成
    temp_dir = tempfile.gettempdir()
    output_path = os.path.join(temp_dir, 'output_synthetic.wav')
    
    # サンプリングレート
    sample_rate = 24000
    
    # テキストの長さに応じた音声の長さを決定
    duration = min(len(text) * 0.15, 15)  # テキストの長さに比例（最大15秒）
    print(f"生成する音声の長さ: {duration:.2f}秒")
    
    # 時間配列の作成
    t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
    
    # 言語に応じて基本周波数を変更
    if language == "ja":
        base_freq = 440  # 日本語 (A4)
    elif language == "en":
        base_freq = 392  # 英語 (G4)
    elif language == "zh":
        base_freq = 330  # 中国語 (E4)
    else:
        base_freq = 262  # その他 (C4)
    
    # 音声波形の初期化
    waveform = np.zeros_like(t)
    
    # テキストの各文字に応じた音を生成し、合成
    for i, char in enumerate(text):
        # 文字ごとに少し周波数を変える
        char_code = ord(char) % 12
        freq = base_freq * (2 ** (char_code / 12))
        
        # 時間オフセット
        start_time = i * duration / max(len(text), 1)
        char_duration = min(duration / max(len(text), 1) * 3, duration - start_time)
        
        # この文字の時間範囲のインデックスを計算
        start_idx = int(start_time * sample_rate)
        end_idx = min(int((start_time + char_duration) * sample_rate), len(t))
        
        # この範囲の時間配列
        t_char = t[start_idx:end_idx] - t[start_idx]
        
        # この文字の音を生成
        char_waveform = 0.5 * np.sin(2 * np.pi * freq * t_char) * np.exp(-t_char / char_duration)
        
        # 全体の波形に加算
        waveform[start_idx:end_idx] += char_waveform
    
    # 正規化
    waveform = waveform / max(abs(waveform.max()), abs(waveform.min())) * 0.9
    
    # 速度調整
    if speed != 1.0:
        # 出力のサンプル数を計算
        output_samples = int(len(waveform) / speed)
        
        # リサンプリング用の入力インデックス
        indices = np.linspace(0, len(waveform) - 1, output_samples)
        
        # 線形補間でリサンプリング
        waveform = np.interp(indices, np.arange(len(waveform)), waveform)
    
    # 音声ファイルとして保存
    wavfile.write(output_path, sample_rate, waveform.astype(np.float32))
    
    print(f"音声ファイルを生成しました: {output_path}")
    return output_path

## 5. ユーティリティ関数の定義

In [ ]:
# MemoryViewからバイトへの変換
def memoryview_to_bytes(mv):
    return bytes(mv)

# バイトからテキストへの変換
def decode_bytes_to_text(content_bytes):
    """バイトデータからテキストを取得する（エンコーディング自動検出）"""
    encodings = ['utf-8', 'shift-jis', 'euc-jp', 'iso-2022-jp']
    
    for encoding in encodings:
        try:
            text = content_bytes.decode(encoding)
            print(f"テキストをエンコーディング {encoding} で読み込みました")
            return text
        except UnicodeDecodeError:
            continue
    
    # どのエンコーディングでも失敗した場合
    raise ValueError("テキストのエンコーディングを検出できませんでした。")

# テキストファイルを読み込む関数
def read_text_file(filepath):
    """テキストファイルを読み込む（エンコーディング自動検出）"""
    encodings = ['utf-8', 'shift-jis', 'euc-jp', 'iso-2022-jp']
    
    for encoding in encodings:
        try:
            with open(filepath, 'r', encoding=encoding) as f:
                text = f.read()
                print(f"ファイルをエンコーディング {encoding} で読み込みました")
                return text
        except UnicodeDecodeError:
            continue
    
    raise ValueError("ファイルのエンコーディングを検出できませんでした。")

## 6. ファイルアップロード機能

In [ ]:
# ファイルアップロードウィジェットの作成
upload_button = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='テキストファイルを選択'
)

# 言語の選択
language_dropdown = widgets.Dropdown(
    options=[('日本語', 'ja'), ('英語', 'en'), ('中国語', 'zh')],
    value='ja',
    description='言語:',
)

# 速度の設定
speed_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.1,
    description='速度:',
    continuous_update=False
)

# 出力表示用
output_area = widgets.Output()
text_preview = widgets.Textarea(
    description='テキスト:',
    placeholder='テキストがここに表示されます',
    disabled=True,
    layout=widgets.Layout(width='100%', height='150px')
)

# ファイルアップロード時の処理
def on_upload_change(change):
    if not change.new:
        return
    
    with output_area:
        output_area.clear_output()
        print("ファイルを処理中...")
        
        try:
            # change.new[0]["content"] からMemoryViewを取得
            content_memoryview = change.new[0]["content"]
            
            # MemoryViewをバイトに変換
            content_bytes = memoryview_to_bytes(content_memoryview)
            
            # バイトからテキストを取得
            text = decode_bytes_to_text(content_bytes)
            
            # テキストプレビューを表示
            text_preview.value = text if len(text) <= 1000 else text[:1000] + "..."
            
            # 言語と速度を取得
            language = language_dropdown.value
            speed = speed_slider.value
            
            # 音声合成
            print(f"テキストを音声に変換中... (言語: {language}, 速度: {speed}x)")
            audio_path = simple_text_to_speech(text, language, speed)
            
            # 音声を表示
            display(Audio(audio_path))
            
            print("処理完了!")
            
        except Exception as e:
            print(f"エラーが発生しました: {str(e)}")

# イベントハンドラの登録
upload_button.observe(on_upload_change, names='value')

# UIの表示
print("テキストファイルをアップロードすると、音声に変換します。")
display(widgets.HBox([upload_button, language_dropdown, speed_slider]))
display(text_preview)
display(output_area)

## 7. 直接テキスト入力機能

In [ ]:
# テキスト入力エリア
input_text = widgets.Textarea(
    placeholder='ここにテキストを入力してください',
    layout=widgets.Layout(width='100%', height='150px')
)

# 言語の選択
input_language = widgets.Dropdown(
    options=[('日本語', 'ja'), ('英語', 'en'), ('中国語', 'zh')],
    value='ja',
    description='言語:',
)

# 速度の設定
input_speed = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.1,
    description='速度:',
    continuous_update=False
)

# 変換ボタン
convert_button = widgets.Button(
    description='テキストを変換',
    button_style='primary'
)

# 出力エリア
input_output = widgets.Output()

# ボタンクリック時の処理
def on_convert_button_click(b):
    text = input_text.value
    if not text:
        with input_output:
            input_output.clear_output()
            print("テキストが入力されていません。")
        return
    
    with input_output:
        input_output.clear_output()
        print("テキストを処理中...")
        
        try:
            # 言語と速度を取得
            language = input_language.value
            speed = input_speed.value
            
            # 音声合成
            print(f"テキストを音声に変換中... (言語: {language}, 速度: {speed}x)")
            audio_path = simple_text_to_speech(text, language, speed)
            
            # 音声を表示
            display(Audio(audio_path))
            
            print("処理完了!")
            
        except Exception as e:
            print(f"エラーが発生しました: {str(e)}")

# イベントハンドラの登録
convert_button.on_click(on_convert_button_click)

# UIの表示
print("テキストを入力して「テキストを変換」をクリックすると、音声に変換します。")
display(input_text)
display(widgets.HBox([input_language, input_speed]))
display(convert_button)
display(input_output)

## 8. サンプルテキストの変換

In [ ]:
# サンプルテキスト
sample_text = """こんにちは。これはFish-Speech-1.5を使った日本語テキスト読み上げのサンプルです。
このモデルは、高品質な日本語音声合成ができます。様々な声色や話速で、自然な発話を生成します。"""

# 言語と速度
language = "ja"
speed = 1.0

# 音声合成
print("サンプルテキストを音声に変換します...")
audio_path = simple_text_to_speech(sample_text, language, speed)

# 音声を表示
display(Audio(audio_path))